In [ ]:
import os
import sys
import json
import numpy as np
import time
import cv2
import skimage
from PIL import Image, ImageDraw
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapOnImage
import csv
%matplotlib inline

In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
#Transforme une vidéos en série d'image.
#DIRECTORY où les vidéos sont enregistrées (Constante)
VIDEO_DIR = ROOT_DIR + 'videos/'
#DIRECTORY où les images seront sauvegardées
IMG_DIR = ROOT_DIR + 'images/'
#Nom de la video (variable)
video_file = '100 NL d Gwandju.MP4'

savePathFolder = IMG_DIR+video_file[:-4]

FOLDER_IMAGE = savePathFolder

dirs = os.listdir(IMG_DIR)

if(video_file[:-4]) in dirs:        
    if not os.path.isdir(savePathFolder):
        try:
            os.mkdir(savePathFolder)
        except:
            print("cant create folder")
else:
    try:
        os.mkdir(savePathFolder)
    except:
        print("cant create folder")
        

capture = cv2.VideoCapture(VIDEO_DIR+video_file)
success,image = capture.read()
count = 0

while success:
  cv2.imwrite(savePathFolder+"/frame%d.jpg" % count, image) 
  success,image = capture.read()
  count += 1

print('save de %d images' %count)

In [ ]:
class NageursConfig(Config):
    """Configuration pour un entrainement fait sur les nageurs.
    Model ispiré de l'original mais adapté à mon GPU
    """
    NAME = "nageur"

    # Le nombre d'image par GPU dépend de la capacité en mémoire
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Nombre de classes
    NUM_CLASSES = 1 + 1  # background + 1 (nageur)

    # Shap
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    #depend de la mémoire
    STEPS_PER_EPOCH = 500

    # This is how often validation is run.
    VALIDATION_STEPS = 5
    
    # Peut être modifié si on souhaite utiliser un autre backbone
    BACKBONE = 'resnet50'

    # Configuration du RPN
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = NageursConfig()
config.display()

In [ ]:
class InferenceConfig(NageursConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    DETECTION_MIN_CONFIDENCE = 0.95
    

inference_config = InferenceConfig()

In [ ]:
# mode détection
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

In [ ]:
#get path pour le dernier entrainement
model_path = model.find_last()

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            #if class_id < 1:
            #    print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
            #    return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            print (image_id)
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    print (image_file_name)
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

In [ ]:
dataset_val = CocoLikeDataset()
dataset_val.load_data(ROOT_DIR + 'datasets/nageur/val/coco_annotations.json', ROOT_DIR + 'datasets/nageur/val/images')
dataset_val.prepare()

In [ ]:
#range les boundings boxes par absisses décroissantes
def TriNageurs (boxes) :
    nageurs = []
    N = boxes.shape[0]
    
    for i in range(0,N):
            y1, x1, y2, x2 = boxes[i]            
            nageurs.append([y1, x1, y2, x2])
    nageursOrdo = []
    yMmax = nageurs[0][0]
    jMax = 0   
    
    for i in range(0,N):        
        yMax = nageurs[0][0]
        jMax = 0
        
        for j in range (0,(N-i)):            
            if (nageurs[j][0] > yMax):
                yMax = nageurs[j][0]
                jMax = j
                
        y1, x1, y2, x2 = nageurs[jMax]        
        nageursOrdo.append([y1, x1, y2, x2])
        nageurs.pop(jMax)
        
    return nageursOrdo

In [ ]:
#Parcours de l'ensemble des images générés depuis la vidéo
real_test_dir = FOLDER_IMAGE
#Tableau contenant toutes les ROIs pour chaque image
fullPos = [[],[]]
files = os.listdir(real_test_dir)
MAX = len(files)
MAX = MAX - int(MAX/10)
frame = 0
for i in range(0, MAX):
    if i%1 == 0:
        if('frame%d.jpg'%i) in files:
            img = skimage.io.imread(real_test_dir+'/'+'frame%d.jpg'%i)
            img_arr = np.array(img)
            results = model.detect([img_arr], verbose=1)
            r = results[0]
            boxes = r['rois']
            N = boxes.shape[0]

            #display_images(np.transpose(activations["res2c_out"][0,:,:,:4], [2, 0, 1]), cols=4)


            if N > 0:
                print(real_test_dir+'/'+'frame%d'%i)
                visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                        dataset_val.class_names, r['scores'], figsize=(5,5))
                print(frame)        
                nageursTries = TriNageurs(boxes)
                print(nageursTries)
                #saveImage(nageursTries,N,img,frame)
                fullPos[0].append(nageursTries)
                fullPos[1].append(frame)
            frame = frame +1

In [ ]:
#Fonction initiant le tracking, cherche des positions pour lesquels on a autant de boundings boxes que de nageurs,
#et des boundings boxes qui se suivent d'une frame à l'autre.
#(on cherche au niveau de la prochaine frame qui à le bon nombre de boundings boxes)
#Renvoi un tableau contenant le début des positions pour un nageur
def SuiviNageur(idNageur, fullPos, posNageur = [[],[]], start = 0):
    count = 0
    delta = 0
    print ('teste à '+str(start))
    for i in range (start, len(fullPos[0])):
        if count < 3: 
            if len(fullPos[0][i]) == 8: 
                k = 1
                count_max = len(fullPos[0]) - i
                for j in range(1, count_max):
                    if len(fullPos[0][i+k]) == 8: 
                        #print ('i :'+str(i))
                        #print('k'+str(i+k))
                        y_a =  fullPos[0][i][idNageur][2]
                        y_b =  fullPos[0][i+k][idNageur][2]
                        x_a =  fullPos[0][i][idNageur][3]
                        x_b =  fullPos[0][i+k][idNageur][3]
                        dist = abs(x_a-x_b)+abs(y_a-y_b)
                        #print(dist)
                        break
                    else:
                        k+=1
                        dist = 1000                        
                if dist < 25+5*k:
                    count += 1
                    if count == 2:
                        delta = i                        
                else:                    
                    delta = 0
                    count = 0
                    #print('r à z : '+str(i))
                    #print("d = "+str(dist))
        else:
            print('reprise à' +str(delta))
            #print(i-1)
            posNageur[0].append(fullPos[0][delta][idNageur])
            posNageur[1].append(fullPos[1][delta])  
            posNageur[0].append(fullPos[0][i-1][idNageur])
            posNageur[1].append(fullPos[1][i-1])
            break
    return posNageur
                

    

In [ ]:
#Récupère le début des positions d'un nageur et le complete pour chaque frame grâce au tableau contenant toutes les positions.
def CompleteNageurPos(posNageur, fullPos, idNageur):
    idStart = posNageur[1][-1]+1
    y_a = posNageur[0][-1][2]
    k = 1
    nageur_perdu = False
    #print(y_a)
    for i in range(len(fullPos[0])):

        if fullPos[1][i]>idStart and nageur_perdu == False:
            distMin = 1000
            jMin = 0
            #si on a créé une position fictive 5 fois de suite, on refait appelle à la fonction d'initialisation pour retrouver la bonne position
            if k%5 == 0:
                posNageur = SuiviNageur(idNageur, fullPos, posNageur,i)
                idStart = posNageur[1][-1]+1
                if idStart == i:
                    nageur_perdu = True                                       
                print('reprise à %d'%idStart)
                y_a = posNageur[0][-1][2]
                k = 1
            else:
                #On associe soit la bounding boxe la plus probable, et si elle n'existe pas, on en créé une en se basant sur les 
                #positions précédentes.
                for j in range(len(fullPos[0][i])):
                    y_b =  fullPos[0][i][j][2]
                    dist = abs(y_a-y_b)
                    if dist < distMin:
                        distMin = dist
                        jMin = j
                if distMin < 30:
                    posNageur[0].append(fullPos[0][i][jMin])
                    posNageur[1].append(fullPos[1][i])                
                    y_a = fullPos[0][i][jMin][2]
                    k=1
                    #print('a la frame '+str(fullPos[1][i])+'ajout de '+str(jMin)+' val '+str(fullPos[0][i][jMin])+'car dMin ='+str(distMin))
                else:
                    print ("données ajoutées "+str(fullPos[1][i])+" car distMin ="+str(distMin))
                    pos1 = posNageur[0][-2]
                    pos2 = posNageur[0][-1]
                    diffPos = [b_elt - a_elt for a_elt, b_elt in zip(pos1, pos2)]
                    newPos = [int((b_elt)/2) + a_elt for a_elt, b_elt in zip(pos2, diffPos)]
                    posNageur[0].append(newPos)
                    posNageur[1].append(fullPos[1][i])
                    k+=1
                    #y_a = newPos[2]
                    #print('new Y ='+str(y_a))

    return posNageur      

In [ ]:
#posNageur.clear()
#posNageur = SuiviNageur(0,fullPos)
#print (posNageur)
#posNageur = CompleteNageurPos(posNageur, fullPos,0)

'''
for i in range(len(posNageur[0])):
    print(posNageur[1][i])    
    print(posNageur[0][i])
    print(fullPos[0][i+16][2])
'''

In [ ]:
#print(posNageur[0][397])

In [ ]:
def ReBuildFullPos (fullPos, nbNageurs):
    newFullPos = []
    for i in range(nbNageurs):
        posNageur = [[],[]]
        posNageur = SuiviNageur(i,fullPos,posNageur)
        posNageur = CompleteNageurPos(posNageur, fullPos,i)
        newFullPos.append(posNageur)
        #posNageur.clear()
    return newFullPos

In [ ]:
newFullPos = ReBuildFullPos (fullPos, 8)
#posNageur = newFullPos[0]
#for i in range (0, 100):
#    print(posNageur[0][i])

In [ ]:
#posNageur = newFullPos[7]
#posNageur1 = newFullPos[2]
#print(posNageur[0])
#for tab in posNageur[0]:
#    for val in tab:
#        if val>1280:
#            print(tab)
#for i in range(1325,1350):
#    print(posNageur0[1][i])
#    print(posNageur0[0][i])
#    print("#####")


In [ ]:
#sauvegarde les positions dans un CSV si on veut les retravailler plus tard ou les garder en mémoire pour ne pas à faire retourner
#le modèle.
def SavePosInCSV (savePathFolder, idNageur, newFullPos):
    posNageur = newFullPos[idNageur]    
    dirs = os.listdir(savePathFolder)
    savePathFolder += "/nageur_"+str(idNageur)
    if("nageur_"+str(idNageur)) in dirs:        
        if not os.path.isdir(savePathFolder):
            try:
                os.mkdir(savePathFolder)
            except:
                print("cant create folder")
    else:
        try:
            os.mkdir(savePathFolder)
        except:
            print("cant create folder")
            
    if os.path.exists(savePathFolder+'/position_nageur_'+str(idNageur)+'.csv'):
        os.remove(savePathFolder+'/position_nageur_'+str(idNageur)+'.csv')
        print('previousFileRemove')

    try:
        with open(savePathFolder+'/position_nageur_'+str(idNageur)+'.csv', 'w', newline="") as f:
            write = csv.writer(f)
            fields = ['frame','x1','x2','y1','y2']
            write.writerow(fields)    

            for i in range(len(posNageur[0])):
                row = []
                row.append(posNageur[1][i])
                row.append(posNageur[0][i][1])
                row.append(posNageur[0][i][3])
                row.append(posNageur[0][i][0])
                row.append(posNageur[0][i][2])
                write.writerow(row)
    except:
        print("error")
            

In [ ]:
savePathFolder = ROOT_DIR + 'datasets/zoom_nageur/100 NL d Gwandju'
nbNageurs = 8
#savePosInCSV(savePathFolder,0,newFullPos)

In [ ]:
def SaveAllPos(savePathFolder,newFullPos,nbNageurs):
    for i in range(nbNageurs):
        SavePosInCSV(savePathFolder,i,newFullPos)

In [ ]:
SaveAllPos(savePathFolder,newFullPos,nbNageurs)

In [ ]:
def showPoseImage(newFullPos, i, nbNageurs, img, idFrame, savePathFolder):
    for idNageur in range (nbNageurs):
        y1, x1, y2, x2 = newFullPos[idNageur][0][i]
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
        pathSave = savePathFolder+'/ROI_'+str(idFrame)+'.jpg'
    cv2.imwrite(pathSave, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))        
    cv2.waitKey(0)

In [ ]:
def saveImage(pos, idNageur, img, idFrame, savePathFolder) :
    savePathFolder += '/nageur_'+str(idNageur)
    y1, x1, y2, x2 = pos
    if y1 < 0:
        y1=0
    if x1 < 0:
        x1=0
    print(idFrame)
    print (pos)

    l = x2 - x1

    if (y1 > 30):
        y1 = y1 - 30

    y2 = y2 +30

    if (x1 > 30):
        x1 = x1 - 30

    x2 = x2 + 30

    imgCrop = img[(y1):(y2),(x1):(x2)]
    #image = cv2.resize(imgCrop,(400,200))
    pathSave = savePathFolder+'/crop_'+str(idFrame)+'.jpg'
    #cv2.imwrite(pathSave, cv2.cvtColor(image, cv2.COLOR_RGB2BGR)) 
    cv2.imwrite(pathSave, cv2.cvtColor(imgCrop, cv2.COLOR_RGB2BGR))        
    cv2.waitKey(0)

In [ ]:
real_test_dir = FOLDER_IMAGE
nbNageurs = 8
image_paths = []
files = os.listdir(real_test_dir)
savePathFolderROI = ROOT_DIR + 'datasets/ROI'

for idNageur in range(nbNageurs):
    print("ajout Frame Nageur %d"%idNageur)
    for idFrame in newFullPos[idNageur][1]:
        i = newFullPos[idNageur][1].index(idFrame)
        pos = newFullPos[idNageur][0][i]      
        if('frame%d.jpg'%idFrame) in files:          
            img = skimage.io.imread(real_test_dir+'/'+'frame%d.jpg'%idFrame)
            saveImage(pos, idNageur, img, idFrame, savePathFolder)
            showPoseImage(newFullPos, i, nbNageurs, img, idFrame, savePathFolderROI)
    

In [ ]:
#Fonctions de créations de vidéos pour la démo.
video_path = ROOT_DIR + 'datasets/zoom_nageur/100 NL d Gwandju/nageur_1/'
width = 260
height = 80
size = (width,height)
out = cv2.VideoWriter(ROOT_DIR + 'datasets/zoom_nageur/100 NL d Gwandju/nageur_1/zoom.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
files = os.listdir(video_path)

for i in range(0,1500):
    if ('crop_'+str(i)+'.jpg') in files :
        img=cv2.resize(cv2.imread(video_path+'crop_'+str(i)+'.jpg'),(width, height))
        out.write(img)
out.release()



In [ ]:
video_path = ROOT_DIR + 'datasets/ROI/'
width = 1280
height = 720
size = (width,height)
out = cv2.VideoWriter(ROOT_DIR + 'datasets/ROI/ROIs.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
files = os.listdir(video_path)

for i in range(0,1500):
    if ('ROI_'+str(i)+'.jpg') in files :
        img=cv2.resize(cv2.imread(video_path+'ROI_'+str(i)+'.jpg'),(width, height))
        out.write(img)
out.release()
